In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [10]:
df = pd.read_csv("cities.csv", dtype="object", encoding="utf-8")
df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,saldanha,-33.0117,17.9442,62.13,79,100,24.94,ZA,1637705348
1,1,iquique,-20.2208,-70.1431,64.44,71,98,4.56,CL,1637705348
2,2,alekseyevsk,57.8406,108.3492,13.86,98,100,2.08,RU,1637705348
3,3,kamaishi,39.2667,141.8833,41.49,93,79,6.2,JP,1637705348
4,4,dikson,73.5069,80.5464,1.13,94,100,12.73,RU,1637705349
...,...,...,...,...,...,...,...,...,...,...
565,565,coueron,47.2151,-1.7217,42.48,73,0,14.97,FR,1637705488
566,566,banjar,-8.19,114.9675,78.66,89,99,4.25,ID,1637705489
567,567,lima,-12.0432,-77.0282,71.91,68,0,12.66,PE,1637705481
568,568,xichang,27.8964,102.2634,43.18,96,100,2.77,CN,1637705345


In [11]:
gmaps.configure(api_key=g_key)

locations = df[["Lat", "Lng"]].astype(float)
humidity = df['Humidity'].astype(float)

In [12]:
fig = gmaps.figure(center = (15,0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=200,
                                 point_radius = 5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [26]:
temp=df['Max Temp'].astype(float)
wind=df['Wind Speed'].astype(float)
cloud=df['Cloudiness'].astype(float)

perfect=df.loc[(temp > 70) & (temp < 85)] 
perfect=perfect.loc[(wind < 10)]
perfect=perfect.loc[(cloud < 1)]
perfect.dropna()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
52,52,mecca,21.4267,39.8261,73.35,50,0,3.4,SA,1637705196
365,365,moron,-34.6534,-58.6198,78.06,34,0,3.0,AR,1637705438
403,403,buin,-33.7333,-70.75,82.74,38,0,9.06,CL,1637705448
435,435,vila velha,-20.3297,-40.2925,75.15,85,0,4.0,BR,1637705456


In [27]:
hotel_df=pd.DataFrame(perfect)
hotel_df['Hotel Name']=" "
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
52,52,mecca,21.4267,39.8261,73.35,50,0,3.4,SA,1637705196,
365,365,moron,-34.6534,-58.6198,78.06,34,0,3.0,AR,1637705438,
403,403,buin,-33.7333,-70.75,82.74,38,0,9.06,CL,1637705448,
435,435,vila velha,-20.3297,-40.2925,75.15,85,0,4.0,BR,1637705456,


In [37]:
url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
params = {"radius": 5000,
         "keyword": "hotel", 
         "key": g_key
         }

for index, row in hotel_df.iterrows():
    lat=row["Lat"]
    lng=row["Lng"]
    city_name=row['City']
    
    params["location"] = f"{lat},{lng}"
    
    address= requests.get(url, params=params).json()
    results = address['results']
    
    try:
        hotel_df.loc[index, "Hotel Name"] = address["results"][0]["name"]
        print(f'Found {address["results"][0]["name"]}')
    except (KeyError, IndexError):
        print("Could Not Find")

In [38]:
hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != ' ']
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
